In [1]:
# Why noy use linear regression

# Logistic hypothesis

# Parameter estimation

### IMPORTAÇÕES NECESSÁRIAS

In [9]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from ipywidgets import interact

%matplotlib widget

In [ ]:
plt.figure(figsize=(5,5))
plt.plot([1,2,3],[2,4,6])

### DESCRIÇÃO DO PROBLEMA

Suponha que temos uma relação de pacientes com câncer de mama e precisamos desenvolver um modelo de Aprendizado de Máquina para prever qual a gravidade do câncer destes pacientes. Entre as respostas possíveis, temos que classificar o câncer como **maligno** ou **benigno**.

Dessa forma, o problema não se encaixa como um problema de regressão, mas sim de classificação. Ou seja, queremos prever uma variável qualitativa, como uma categoria (**maligno** ou **benigno**).

Para que os computadores possam "entender" as variáveis qualitativas, é preciso traduzi-las para a linguagem numérica, ex.: benigno = 0; e maligno = 1. Este processo de tradução é chamado de ***encoding*** ou **codificação**.

Veremos mais à frente que estes valores qualitativos podem ser interpretados como a probabilidade de uma determinada amostra pertencer a classe-alvo.

### CARREGAMENTO E VISUALIZAÇÃO DOS DADOS

In [ ]:
data = load_breast_cancer(as_frame=True)

print(f"{'-'*80}\nVariáveis independentes: {data['feature_names']} \n{'-'*80}\nVariáveis dependentes: {data['target_names']}")

In [ ]:
print(f'{"-"*100} \nX:')
display(data['data'].head(5))

print(f'{"-"*100} \ny:')
display(data['target'][-10:])

Ao analisar a base de dados, observa-se que o valor correspondente à classe **maligno** é 0 e aquele correspondente à classe **benigno** é 1. É comum em um problema de classificação binária (duas classes-alvo) que a classe principal a qual se deseja prever seja representada pelo número 1. Dessa forma, por questões didáticas, decidi trocar os valores entre as classes. Ou seja, agora o valor referente à classe **maligno** (classe principal) será 1 e o da classe **benigno** será 0.

**obs: essa mudança de ordem não altera em nada o processo de desenvolvimento e otimização do modelo de regressão logística.**

In [5]:
X, y = data['data'].values, data['target'].values

y = np.where(y == 0, 1, 0)

In [ ]:
fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='benigno')
ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='maligno')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.legend()

fig.tight_layout()
plt.show()

### POR QUE NÃO USAR REGRESSÃO LINEAR?

Uma vez transformadas as classes em valores binários 0/1, podemos treinar um regressor linear para estimar qual a categoria do câncer de um determinado paciente da seguinte forma:

In [ ]:
x_ = np.linspace(X[:,2].min(), X[:,2].max(), 10)
y_ = x_ * (1 / 40) - 2 # Linear Regressor

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0.plot(x_, y_, linestyle='--', c='black', label='Regressor Linear')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.legend()

fig.tight_layout()
plt.show()

Analisando a figura anterior, podemos interpretar os valores preditos pelo regressor linear como uma estimativa de $P(y=1|X)$. Isto é, a probabilidade de uma determinada amostra $X$ pertencer a classe 1 (maligno). Observa-se que os valores estimados pelo regressor linear ultrapassam os limites de probabilidae (0 e 1) e podem variar de $-\infty$ a $+\infty$.

Portanto, é por essas e outras razões que é preferível a utilização de um regressor mais adequado capaz de lidar com valores qualitativos, como o modelo de **Regressão Logística**

### REGRESSÃO LOGÍSTICA

O modelo de Regressão Logística tem esse nome devido à função que serve de hipótese para o modelo: função logística, mais especificamente a função sigmóide.

A imagem da função sigmóide varia entre os valores 0 e 1. Sua equação é descrita da seguinte forma:

$$g(z) = \frac{1}{1 + e^{-z}}$$

Veremos mais à frente que, quando utilizada para modelar um problema de classificação do tipo $P(y=1|X)$, a função precisa ser ajustada e $z$ passa a ser uma abreviação de $w \cdot X + b$ e a versão completa da função é, então:

$$ g_{W, b}(X) = \frac{1}{1 + e^{-(W \cdot X + b)}} $$

onde $g_{W, b}(X) = P(y=1|X)$ é a variável dependente (saída), $X$ são as variáveis independentes (entrada), $W$ e $b$ são parâmetros a serem otimizados.

**OBS: para facilitar, a partir de agora, vamos abreviar $g_{W, b}(X)$ por $\hat{y}$, onde entende-se que $g(.)$ é a resposta calculada pelo modelo ($g(.)=\hat{y}$)**

A forma em "S" da função sigmoide pode ser visualizada na figura a seguir:

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-10, 10, 100)
g = sigmoid(z)

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.plot(z, g, c='black', label='Sigmoid')
ax0.set_xlabel('$z$')
ax0.set_ylabel('$g(z)$')
ax0.legend()

fig.tight_layout()
plt.show()

Agora vamos tentar implementar a função logística para receber como entrada as variáveis do nosso problema de classificação de câncer de mama e ver como os parâmetros influenciam a função.

In [10]:
# utilizar utils

def standardize(x):
    return (x - x.mean(axis=0)) / x.std(axis=0)

In [11]:
X = standardize(X)

In [ ]:
x_ = np.linspace(X[:,2].min(), X[:,2].max(), 1000)

fig = plt.figure(figsize=(5,4))
ax0 = fig.add_subplot(1,1,1)

def update(w, b):

    z = w * x_ + b
    g = sigmoid(z)

    ax0.clear()
    ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
    ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
    ax0.plot(x_, g, linestyle='--', c='black', label=f'w={round(w, 2)} | b={round(b, 2)}')
    ax0.set_xlabel(data['feature_names'][2])
    ax0.set_ylabel('Class')
    ax0.legend()

interact(update, w=widgets.FloatSlider(value=0, min=-10, max=10), b=widgets.FloatSlider(value=0, min=-10, max=10))

fig.tight_layout()
plt.show()

### OTIMIZAÇÃO DOS PARÂMETROS

Quando desejamos modelar um regressor logístico para um problema de classificação, estimamos uma função logística cujos coeficientes $W$ e $b$ são desconhecidos. Portanto, é necessário estimar os coeficientes de maneira que a curva melhor se ajuste aos dados de treinamento.

Para a otimização dos coeficientes do modelo de Regressão Linear, é utilizado o método dos Mínimos Quadrados para minimizar a diferença entre os dados preditos $\hat{y}$ e reais $y$ da seguinte forma:

$$ MSE = \frac{1}{2m} \sum_{i=0}^{m-1} (\hat{y_{i}} - y_{i})^{2} $$

A utilização da MSE como função custo da Regressão Logística não é indicada e abaixo demonstraremos o porquê:

In [13]:
def mse(y_pred, y):
    return np.mean((y_pred - y)**2, axis=0) / 2

In [ ]:
n_grid = 25

w_ = np.linspace(-25, 25, n_grid).reshape(1,-1)
b_ = np.linspace(-25, 25, n_grid).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

x_ = X[:, 2].reshape(-1,1)
z_ = np.dot(x_, W_.reshape(1,-1)) + B_.reshape(1,-1)
y_pred = sigmoid(z_)

mse_ = mse(y_pred, np.tile(y.reshape(-1,1), n_grid**2))
mse_ = mse_.reshape(n_grid, n_grid)

fig = plt.figure(figsize=(5,5))

ax0 = fig.add_subplot(111, projection='3d')
ax0.plot_surface(W_, B_, mse_, cmap='viridis')
ax0.set_xlabel('$W$')
ax0.set_ylabel('$b$')
ax0.set_zlabel('$MSE$')

fig.tight_layout()
plt.show()

Devido ao fato de a função logística (sigmoide) ser não-linear, a função MSE, quando utilizada no contexto da Regressão Logística, assume uma forma não-convexa. Isto é, existem vários pontos da função custo diferentes do mínimo global que também possuem derivadas parciais com relação aos parâmetros iguais a 0. Dessa forma, a otimização dos parâmetros através do metódo Gradiente Descendente é prejudicada, uma vez que quando o gradiente é nulo não há atualização.

Ao invés de utilizar a MSE, é utilizada uma abordagem diferente: da Probabilidade Máxima (*Maximum Likelihood*). A estimativa dos coeficientes de um modelo de Regressão Logística é dada de tal forma a maximizar as probabilidades de classificação em cada classe.

$ \begin{equation}
    \begin{align*}
        \text{Likelihood} = L(y, \hat{y}) & = \prod_{i=0}^{m-1} P(y_{i}=1|X_{i})^{y_{i}} \prod_{i=0}^{m-1} P(y_{i}=0|X_{i})^{1-y_{i}} \\
        & = \prod_{i=0}^{m-1} P(y_{i}=1|X_{i})^{y_{i}} \prod_{i=0}^{m-1} \left[ 1 - P(y_{i}=1|X_{i}) \right]^{1-y_{i}} \\
        & = \prod_{i=0}^{m-1} \hat{y_{i}}^{y_{i}} \prod_{i=0}^{m-1} \left( 1 - \hat{y_{i}} \right)^{1-y_{i}}
    \end{align*}
\end{equation} $

Através do produtório das probabilidades preditas para cada classe, a função *Likelihood* mede o nível de acerto do modelo em uma escala de probabilidade $[0, 1]$. Na prática, a probabilidade predita ($\hat{y}$) nunca será igual a 0 ou 1, de fato, porque a função sigmoid nunca alcança esses valores. Entretanto, quanto mais perto as probabilidades preditas forem das verdadeiras, maior será o valor do *Likelihood*.

In [15]:
def likelihood(y, y_hat):
    return np.prod(y_hat**y * (1 - y_hat)**(1-y), axis=0)

In [ ]:
x = X[:,2].reshape(-1,1)
x_ = np.linspace(x.min(), x.max(), 1000)

N = 25

w_max = b_max = 10
w_min = b_min = -w_max

w_ = np.linspace(w_min, w_max, N).reshape(1,-1)
b_ = np.linspace(b_min, b_max, N).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

Z_ = np.dot(x, W_.reshape(1,-1)) + B_.reshape(1,-1)
Y_hat_ = sigmoid(Z_)

L_ = likelihood(np.tile(y.reshape(-1,1), N**2), Y_hat_)
L_ = L_.reshape(N, N)

fig = plt.figure(figsize=(8,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, projection='3d')

ax0.scatter(x[y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(x[y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0_line, = ax0.plot(x_, x_, linestyle='--', c='black')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.set_ylim(-.1, 1.1)
ax0.legend()

ax1.plot_surface(W_, B_, L_, cmap='viridis', zorder=0)
ax1_line, = ax1.plot([], [], [], marker='o', color='orange', zorder=3)
ax1.set_xlabel('$W$')
ax1.set_ylabel('$b$')
ax1.set_zlabel('Likelihood')

def update(w, b):

    z_ = w * x_ + b
    z = w * x + b
    y_hat_ = sigmoid(z_)
    y_hat = sigmoid(z)
    lh = likelihood(y, y_hat.flatten())
    
    ax0_line.set_ydata(y_hat_)
    ax0_line.set_label(f'w:{w} | b:{b}')
    ax0.set_title(f'Likelihood: {lh}')

    ax1_line.set_data([w], [b])
    ax1_line.set_3d_properties(lh)

interact(update, w=widgets.FloatSlider(value=1, min=w_min, max=w_max), b=widgets.FloatSlider(value=0, min=b_min, max=b_max))

fig.tight_layout()
plt.show()

Observando os resultados da função *Likelihood*, tem-se que, devido ao produtório de uma série de valores menores que 1 e maiores que 0, o valor da função é, para a maioria dos coeficientes, próximo de zero e, portanto, métodos que dependem do gradiente para a atualização dos coeficientes não funcionarão.

Para contornar esse problema, a função logarítmica é aplicada à função *Likelihood*: 

$$ \begin{equation*}
    log \left(L(y, \hat{y}) \right) = log\left(\prod_{i=0}^{m-1} \hat{y_{i}}^{y_{i}} \prod_{i=0}^{m-1} \left( 1 - \hat{y_{i}} \right)^{1-y_{i}}\right)
\end{equation*} $$

Aplicando a seguinte propriedade do logarítmo: $log(A \cdot B)=log(A)+log(B)$ e chamando $log(L(y, \hat{y}))$ de $LL(y, \hat{y})$ temos que

$$ \begin{equation*}
    \begin{align*}
        LL(y, \hat{y}) & = \sum_{i=0}^{m-1} log(\hat{y_{i}}^{y_{i}}) + log([1 - \hat{y_{i}}]^{1 - y_{i}}) \\
        & = \sum_{i=0}^{m-1} y_{i} \cdot log(\hat{y_{i}}) + (1 - y_{i}) log([1 - \hat{y_{i}}]) \\
    \end{align*}
\end{equation*} $$

Com isso, temos a expressão para o *log-likelihood*. Sua forma pode ser visualizada na figura abaixo:

In [17]:
def log_likelihood(y, y_hat):
    return np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat), axis=0)

In [ ]:
x = X[:,2].reshape(-1,1)
x_ = np.linspace(x.min(), x.max(), 1000)

N = 25

w_max = b_max = 10
w_min = b_min = -w_max

w_ = np.linspace(w_min, w_max, N).reshape(1,-1)
b_ = np.linspace(b_min, b_max, N).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

Z_ = np.dot(x, W_.reshape(1,-1)) + B_.reshape(1,-1)
Y_hat_ = sigmoid(Z_)

L_ = log_likelihood(np.tile(y.reshape(-1,1), N**2), Y_hat_)
L_ = L_.reshape(N, N)

fig = plt.figure(figsize=(8,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, projection='3d')

ax0.scatter(x[y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(x[y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0_line, = ax0.plot(x_, x_, linestyle='--', c='black')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.set_ylim(-.1, 1.1)
ax0.legend()

ax1.plot_surface(W_, B_, L_, cmap='viridis', zorder=0)
ax1_line, = ax1.plot([], [], [], marker='o', color='orange', zorder=3)
ax1.set_xlabel('$W$')
ax1.set_ylabel('$b$')
ax1.set_zlabel('Likelihood')

def update(w, b):

    z_ = w * x_ + b
    z = w * x + b
    y_hat_ = sigmoid(z_)
    y_hat = sigmoid(z)
    lh = log_likelihood(y, y_hat.flatten())
    
    ax0_line.set_ydata(y_hat_)
    ax0_line.set_label(f'w:{w} | b:{b}')
    ax0.set_title(f'Likelihood: {lh}')

    ax1_line.set_data([w], [b])
    ax1_line.set_3d_properties(lh)

interact(update, w=widgets.FloatSlider(value=1, min=w_min, max=w_max), b=widgets.FloatSlider(value=0, min=b_min, max=b_max))

fig.tight_layout()
plt.show()

Para a otimização dos coeficientes da Regressão linear é comum utilizar métodos de minimização de funções, em detrimento daqueles de maximização. Portanto, na realidade, a função cuja **minimização** significa encontrar os melhores coeficientes (função custo) é o negativo do *log-likelihood*:

$$ -L(y, \hat{y}) = J(y, \hat{y}) = \frac{1}{m} \sum_{i=1}^{m-1} -y_{i} \cdot log(\hat{y_{i}}) -(1 - y_{i}) \cdot log(1 - \hat{y_{i}}) $$

A forma de $J(y, \hat{y})$ pode ser visualizada na figura abaixo:

In [19]:
def log_loss(y, y_hat):
    return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat), axis=0)

In [ ]:
x = X[:,2].reshape(-1,1)
x_ = np.linspace(x.min(), x.max(), 1000)

N = 25

w_max = b_max = 10
w_min = b_min = -w_max

w_ = np.linspace(w_min, w_max, N).reshape(1,-1)
b_ = np.linspace(b_min, b_max, N).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

Z_ = np.dot(x, W_.reshape(1,-1)) + B_.reshape(1,-1)
Y_hat_ = sigmoid(Z_)

L_ = log_loss(np.tile(y.reshape(-1,1), N**2), Y_hat_)
L_ = L_.reshape(N, N)

fig = plt.figure(figsize=(8,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, projection='3d')

ax0.scatter(x[y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(x[y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0_line, = ax0.plot(x_, x_, linestyle='--', c='black')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.set_ylim(-.1, 1.1)
ax0.legend()

ax1.plot_surface(W_, B_, L_, cmap='viridis', zorder=0)
ax1_line, = ax1.plot([], [], [], marker='o', color='orange', zorder=3)
ax1.set_xlabel('$W$')
ax1.set_ylabel('$b$')
ax1.set_zlabel('Likelihood')

def update(w, b):

    z_ = w * x_ + b
    z = w * x + b
    y_hat_ = sigmoid(z_)
    y_hat = sigmoid(z)
    lh = log_loss(y, y_hat.flatten())
    
    ax0_line.set_ydata(y_hat_)
    ax0_line.set_label(f'w:{w} | b:{b}')
    ax0.set_title(f'Likelihood: {lh}')

    ax1_line.set_data([w], [b])
    ax1_line.set_3d_properties(lh)

interact(update, w=widgets.FloatSlider(value=1, min=w_min, max=w_max), b=widgets.FloatSlider(value=0, min=b_min, max=b_max))

fig.tight_layout()
plt.show()

Após a obtenção da função custo para a Regressão Logística, podemos observar que a função resultante é agora convexa e suave. Além de os valores estarem adequados, nem muito pequenos, nem muito grandes.

Encontrar os parâmetros que minimizem essa função significa encontrar aqueles que satisfaçam a equação:

$$ \nabla_{w,b} J(y, \hat{y}) = \frac{\partial J}{\partial w}\textbf{i} + \frac{\partial J}{\partial b}\textbf{j} = 0 \Rightarrow \begin{cases}
                \frac{\partial J}{\partial w} = 0 \\ \\
                \frac{\partial J}{\partial b} = 0
            \end{cases} $$

Para isso, então, é necessário calcular as derivadas parciais da função custo com relação aos coeficientes $w$ e $b$.

Primeiro, vamos calcular a derivada com relação a $w$:

$$\begin{align*}
    \frac{\partial J}{\partial w} & = \frac{\partial}{\partial w} \left[\frac{1}{m} \sum_{i=0}^{m-1} - y_{i} \cdot log(\hat{y_{i}}) - (1 - y_{i}) \cdot log(1 - \hat{y_{i}}) \right] \\
    & = \frac{1}{m} \sum_{i=0}^{m-1} - y_{i} \cdot \frac{\partial}{\partial w} log(\hat{y_{i}}) - (1 - y_{i}) \cdot \frac{\partial}{\partial w} log(1 - \hat{y_{i}})
\end{align*}$$

#### (1) Calculando $\frac{\partial}{\partial w} log(\hat{y_{i}})$

$$ \frac{\partial}{\partial w} log(\hat{y_{i}}) = \frac{\partial}{\partial w} log\left(\frac{1}{1 + e^{wx+b}}\right) $$

Para isso, vamos fazer as seguintes substituições de variáveis:

$$ \begin{cases}
    g = log(\frac{1}{1 + e^{wx + b}}) = log(\hat{y}) \\
    k = \frac{1}{1 + e^{wx + b}} = \hat{y} \\
    v = 1 + e^{wx + b} = \hat{y}^{-1} \\
    u = wx + b
\end{cases} $$

Dessa forma, de acordo com a regra da cadeia, temos que $\frac{\partial g}{\partial w} = \frac{\partial g}{\partial k} \cdot \frac{\partial k}{\partial v} \cdot \frac{\partial v}{\partial u} \cdot \frac{\partial u}{\partial w}$.

Resolvendo as derivadas parciais, temos

$$ \begin{cases}
    \frac{\partial g}{\partial k} = \frac{1}{k} = \hat{y}^{-1} \\
    \frac{\partial k}{\partial v} = -v^{-2} = -\hat{y}^{2} \\
    \frac{\partial v}{\partial u} = e^{u} = \hat{y}^{-1} - 1 \\
    \frac{\partial u}{\partial w} = x
\end{cases} $$

Substituindo, temos

$$ \begin{align*}
    \frac{\partial g}{\partial w} & = \frac{\partial g}{\partial k} \cdot \frac{\partial k}{\partial v} \cdot \frac{\partial v}{\partial u} \cdot \frac{\partial u}{\partial w} \\
    & = \hat{y}^{-1} \cdot (-\hat{y}^{2}) \cdot (\hat{y}^{-1} - 1) \cdot x \\
    & = -\hat{y} \cdot (\hat{y}^{-1} - 1) \cdot x \\
    & = (\hat{y} - 1) \cdot x
\end{align*} $$

#### (2) Calculando $\frac{\partial}{\partial w} log(1 - \hat{y_{i}})$

$$ \frac{\partial}{\partial w} log( 1 - \hat{y_{i}}) = \frac{\partial}{\partial w} log\left(1 - \frac{1}{1 + e^{wx+b}}\right) $$

Para isso, vamos fazer as seguintes substituições de variáveis:

$$ \begin{cases}
    g = log\left(1 - \frac{1}{1 + e^{wx+b}}\right) = log(1 - \hat{y}) \\
    k = 1 - \frac{1}{1 + e^{wx + b}} = 1 - \hat{y} \\
    v = 1 + e^{wx + b} = \hat{y}^{-1} \\
    u = wx + b
\end{cases} $$

Dessa forma, de acordo com a regra da cadeia, temos que $\frac{\partial g}{\partial w} = \frac{\partial g}{\partial k} \cdot \frac{\partial k}{\partial v} \cdot \frac{\partial v}{\partial u} \cdot \frac{\partial u}{\partial w}$.'

Resolvendo as derivadas parciais, temos

$$ \begin{cases}
    \frac{\partial g}{\partial k} = \frac{1}{k} = \frac{1}{1 - \hat{y}} \\
    \frac{\partial k}{\partial v} = v^{-2} = \hat{y}^{2} \\
    \frac{\partial v}{\partial u} = e^{u} = \hat{y}^{-1} - 1 \\
    \frac{\partial u}{\partial w} = x
\end{cases} $$

Substituindo, temos

$$ \begin{align*}
    \frac{\partial g}{\partial w} & = \frac{\partial g}{\partial k} \cdot \frac{\partial k}{\partial v} \cdot \frac{\partial v}{\partial u} \cdot \frac{\partial u}{\partial w} \\
    & = \frac{1}{1 - \hat{y}} \cdot \hat{y}^{2} \cdot (\hat{y}^{-1} - 1) \cdot x \\
    & = \frac{\hat{y} \cdot (1 - \hat{y})}{1 - \hat{y}} \cdot x \\
    & = \hat{y} \cdot x
\end{align*} $$

Substituindo os resultados de $\frac{\partial}{\partial w} log(\hat{y_{i}})$ e $\frac{\partial}{\partial w} log(1 - \hat{y_{i}})$ em $\frac{\partial J}{\partial w}$, temos

$$\begin{align*}
    \frac{\partial J}{\partial w} & = \frac{1}{m} \sum_{i=0}^{m-1} - y_{i} \cdot \frac{\partial}{\partial w} log(\hat{y_{i}}) - (1 - y_{i}) \cdot \frac{\partial}{\partial w} log(1 - \hat{y_{i}}) \\
    & = \frac{1}{m} \sum_{i=0}^{m-1} - y_{i} \cdot (\hat{y}_{i} - 1) \cdot x_{i} - (1 - y_{i}) \cdot \hat{y}_{i} \cdot x_{i} \\
    & = \frac{1}{m} \sum_{i=0}^{m-1} [- y_{i} \cdot (\hat{y}_{i} - 1) - (1 - y_{i}) \cdot \hat{y}_{i}] \cdot x_{i} \\
    & = \frac{1}{m} \sum_{i=0}^{m-1} (-y_{i} \hat{y_{i}} + y_{i} - \hat{y}_{i} + y_{i} \hat{y}_{i}) \cdot x_{i} \\
    & = \frac{1}{m} \sum_{i=0}^{m-1} (y_{i} - \hat{y}_{i}) \cdot x_{i}
\end{align*}$$

Fazendo os mesmos passos anteriores para $\frac{\partial J}{\partial b}$ temos ao final:

$$ \begin{cases}
    \frac{\partial J}{\partial w} = (y_{i} - \hat{y}_{i}) \cdot x_{i} \\ \\
    \frac{\partial J}{\partial b} = (y_{i} - \hat{y}_{i})
\end{cases} $$

As equações das derivadas parciais obtidas com relação ao valor predito $\hat{y}$ são idênticas às da Regressão Linear. Entretanto, diferentemente da Regressão Linear, a equação $\nabla_{w,b} J = 0$ não tem solução analítica para os coeficientes $w$ e $b$ devido à não-linearidade da função logística.

Assim, são necessários métodos iterativos para a atualização dos coeficientes. O algoritmo de otimização mais comum é o Gradiente Descente, cujas equações são definidas por:

$$\begin{cases}
    w_{t + 1} = w_{t} - \alpha \cdot \frac{\partial J}{\partial w} \\ \\
    b_{t + 1} = b_{t} - \alpha \cdot \frac{\partial J}{\partial b}
\end{cases}$$

Agora, vamos implementar o código para atualização dos coeficientes

#### TREINAMENTO

In [59]:
def compute_gradients(X, y, y_hat):
    m, n = X.shape
    err = y - y_hat
    print(err.shape, X.shape, y.shape, y_hat.shape)
    dj_dw = np.dot(err.reshape(1,-1), X) / m
    dj_db = np.mean(err, axis=0)
    return dj_dw, dj_db

In [62]:
def fit(X, y, w, b, learning_rate=0.001, epochs=100):
    history = {'epoch':[], 'loss':[], 'w':[], 'b':[]}
    for epoch in range(epochs):
        print(w)
        z = np.dot(X, w) + b
        y_hat = sigmoid(z)

        dj_dw, dj_db = compute_gradients(X, y, y_hat)
        w = w + learning_rate * dj_dw
        b = b + learning_rate * dj_db

        loss = log_loss(y, y_hat.flatten())

        if epoch % 10 == 0:
            print(f'Loss: {loss}')

        history['epoch'] += [epoch]
        history['loss'] += [loss]
        history['b'] += [w]
        history['w'] += [b]
    return history

In [ ]:
x = X[:,2].reshape(-1,1)
w, b = np.zeros(1), 0

history = fit(x, y, w, b, 1, 100)

In [ ]:
x_ = np.linspace(x.min(), x.max(), 1000)


In [ ]:
x = X[:,2].reshape(-1,1)
x_ = np.linspace(x.min(), x.max(), 1000)

N = 25

w_max = b_max = 10
w_min = b_min = -w_max

w_ = np.linspace(w_min, w_max, N).reshape(1,-1)
b_ = np.linspace(b_min, b_max, N).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

Z_ = np.dot(x, W_.reshape(1,-1)) + B_.reshape(1,-1)
Y_hat_ = sigmoid(Z_)

L_ = log_loss(np.tile(y.reshape(-1,1), N**2), Y_hat_)
L_ = L_.reshape(N, N)

fig = plt.figure(figsize=(8,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, projection='3d')

ax0.scatter(x[y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(x[y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0_line, = ax0.plot(x_, x_, linestyle='--', c='black')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.set_ylim(-.1, 1.1)
ax0.legend()

ax1.plot_surface(W_, B_, L_, cmap='viridis', zorder=0)
ax1_line, = ax1.plot([], [], [], marker='o', color='orange', zorder=3)
ax1.set_xlabel('$W$')
ax1.set_ylabel('$b$')
ax1.set_zlabel('Likelihood')

def update(w, b):

    z_ = w * x_ + b
    z = w * x + b
    y_hat_ = sigmoid(z_)
    y_hat = sigmoid(z)
    lh = log_loss(y, y_hat.flatten())
    
    ax0_line.set_ydata(y_hat_)
    ax0_line.set_label(f'w:{w} | b:{b}')
    ax0.set_title(f'Likelihood: {lh}')

    ax1_line.set_data([w], [b])
    ax1_line.set_3d_properties(lh)

interact(update, w=widgets.FloatSlider(value=1, min=w_min, max=w_max), b=widgets.FloatSlider(value=0, min=b_min, max=b_max))

fig.tight_layout()
plt.show()